# Valoración opciones aplicada

Tome en consideración el modelo continuio de Black and Scholes:

$$C(S,t)=S\cdot\Phi(d_1)-K\exp(-R\cdot t)\cdot\Phi(d_2)$$

donde $d_1=\frac{\log\frac{S}{K}+\left(R+\frac{\sigma^2}{2}\right)\cdot t}{\sqrt{\sigma^2\cdot t}}$ y $d_2=d_1-\sqrt{\sigma^2\cdot t}$.

Defina los parámetros asociados a valores del activo definido en la Tarea 1 y calcule el valor de una opción de compra.

## Black-Scholes

In [2]:
import math
from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    phi1 = norm.cdf(d1)
    phi2 = norm.cdf(d2)
    precio_call = S * phi1 - K * math.exp(-r * T) * phi2
    return precio_call

def black_scholes_put(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    precio_put = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return precio_put

          # Parámetros de test
          #K = 100
          #r = 0.02
          #sigma = 0.2
          #T = 0.5
          #S0 = 102
          # Resultado esperado CALL
          # 7.28815118277163

#Parametros de ENELCHILE desde API bolsa de santiago
K = 45
r = 6.6
sigma = 0.75
T = 1
S0 = 53.90

precio = black_scholes_call(S0, K, T, r, sigma)
print(f"Precio de la opción de compra call: {precio}")

precio = black_scholes_put(S0, K, T, r, sigma)
print(f"Precio de la opción de compra put: {precio}")

Precio de la opción de compra call: 53.838783438310344
Precio de la opción de compra put: 1.0704837295845274e-20


## Segundo Modelo de valoración

Modelo Binomial

El modelo binomial permite analizar el comportamiento de las acciones a lo largo del tiempo. Recibe su nombre de binomia" debido a la suposición de que el precio de una acción en un momento t dado, puede evolucionar de dos maneras distintas. Puede subir en el momento t+1 y ser evaluado como Su, o puede bajar y ser evaluado como Sd. La probabilidad de que ocurra una subida es denotada por p, mientras que la probabilidad de una bajada se representa como (1-p).

$ Precio_{t} = \frac{{p \cdot Precio_{t+1u} + (1-p) \cdot Precio_{t+1d}}}{{R}} $

Donde:
- $( Precio_{t} $) es el valor del nodo en el tiempo t.
- $( p $) es la probabilidad de subida.
- $( Precio_{t+1u} $) es el valor del nodo en el tiempo t+1 en caso de una subida.
- $( (1-p) $) es la probabilidad de bajada.
- $( Precio_{t+1d} $) es el valor del nodo en el tiempo t+1 en caso de una bajada.
- $( R $) es el factor de descuento.

Cuando en el modelo binomial , los ensayos (n) tienden al infinito, bajo ciertas condiciones, es posible que el modelo binomial se transforme en el modelo de Black-Scholes en la valoracion de acciones financieras.

In [3]:
def binomial(S, K, T, r, sigma, n, option_type="call"):
    dt = T / n
    u = math.exp(sigma * math.sqrt(dt))
    d = 1 / u
    p = (math.exp(r * dt) - d) / (u - d)

    precios = [[0.0 for i in range(n + 1)] for i in range(n + 1)]


    for j in range(n + 1):     # Precio del activo subyacente en los ultimos nodos
        precios[n][j] = S * (u ** (n - j)) * (d ** j)


    for j in range(n + 1): # Valor de la opcion en el ultimo nodo
        if option_type == "call":
            precios[n][j] = max(0, precios[n][j] - K)
        else:
            precios[n][j] = max(0, K - precios[n][j]) ##Cuando el tipo de opcion es put


    for i in range(n - 1, -1, -1):   # Valor de la opcion desde los ultimos nodos
        for j in range(i + 1):
            precios[i][j] = max(0, math.exp(-r * dt) * ((p * precios[i + 1][j]) + ((1 - p) * precios[i + 1][j + 1])))

    return precios[0][0]

n = 1000  # Número de pasos en el modelo binomial (No usar mas de 10000 porque google collab se cae)

precio_call = binomial(S0, K, T, r, sigma, n, option_type="call")
precio_put = binomial(S0, K, T, r, sigma, n, option_type="put")

print(f"Precio de la opción de compra call: {precio_call}")
print(f"Precio de la opción de compra put: {precio_put}")

Precio de la opción de compra call: 53.83878343831542
Precio de la opción de compra put: 1.9832379218846802e-21


## Comparacion de ambos modelos

Diferencias:
El modelo Black-Scholes se trabaja de manera continua, mientras que el modelo Binomial trabaja en tiempo discreto. Esto significa que el Black-Scholes asume cambios continuos en los precios de los activos, mientras que el Binomial representa los movimientos de precios en pasos discretos a lo largo del tiempo. Además, el modelo Binomial requiere más tiempo de cálculo ya que se debe construir un árbol de precios y calcular todos los escenarios posibles en función de la cantidad de períodos de tiempo, a diferencia del método de Black-Scholes que es más directo. Además, el modelo Black-Scholes se basa en parámetros fijos, mientras que el modelo Binomial permite la variación de parámetros a medida que se analizan los diferentes periodos de tiempo.

Similitudes:
Ambos modelos, Black-Scholes y Binomial, son aceptados como herramientas para evaluar y predecir los precios de opciones. Ambos modelos trabajan con parámetros comunes relacionados con la opción, como el precio del activo subyacente, el precio de ejercicio, la volatilidad y el tiempo hasta el vencimiento. A pesar de sus diferencias en enfoque y complejidad, estos modelos se utilizan en el campo de las finanzas para comprender y valorar opciones, y comparten la misma base teórica de la teoría de opciones financieras.